## Knitro VS Optizelle 1

Simple script where we compare Knitro and Optizelle solving the same two problems

In [1]:
import os
import os.path as osp
elastic_rods_dir = '../../../elastic_rods/python/'
weaving_dir = '../../'
import sys
sys.path.append(elastic_rods_dir)
sys.path.append(weaving_dir)
import numpy as np
import elastic_rods
import linkage_vis
from bending_validation import suppress_stdout as so
from elastic_rods import EnergyType, InterleavingType

## Input parameters

In [2]:
rod_length = 0.3534025445286393
width = rod_length / 15 * 5 / 2
thickness = width / 5 * 0.35 * 5

In [3]:
# Sphere 1
default_camera_parameters = ((3.466009282140468, -4.674139805388271, -2.556131049738206),
                             (-0.21402574298422497, -0.06407538766530313, -0.9747681088523519),
                             (0.1111, 0.1865, 0.5316))
RIBBON_CS = [thickness, width]
MODEL_NAME = "sphere_1"
MODEL_PATH = osp.join(weaving_dir + 'normalized_objs/models/{}.obj'.format(MODEL_NAME))
SUBDIVISION_RESOLUTION = 20
INPUT_SURFACE_PATH = osp.join(weaving_dir + 'normalized_objs/surface_models/{}.obj'.format(MODEL_NAME))

In [4]:
minRestLen = RIBBON_CS[1]*2.0

## Optimizer Parameters

In [5]:
OPTS = elastic_rods.NewtonOptimizerOptions()
OPTS.gradTol = 1e-6
OPTS.verbose = 10;
OPTS.beta = 1e-8
OPTS.niter = 100
OPTS.verboseNonPosDef = False

In [6]:
rw = 0.1 * 5
sw = 0.1 * 5

## Helper functions

### Initialize Woven Linkage

There are two different types of linkage: the `RodLinkage` and the `SurfaceAttractedLinkage`. The `SurfaceAttractedLinkage` has the additional surface attraction weight and hence need the `surface_path` as parameter.

In [7]:
def initialize_linkage(surface_path=INPUT_SURFACE_PATH, 
                       useCenterline=True, 
                       cross_section=RIBBON_CS, 
                       subdivision_res=SUBDIVISION_RESOLUTION, 
                       model_path=MODEL_PATH):
    l = elastic_rods.SurfaceAttractedLinkage(surface_path, 
                                             useCenterline, 
                                             model_path, 
                                             subdivision_res, 
                                             False, 
                                             InterleavingType.triaxialWeave)
    l.setMaterial(elastic_rods.RodMaterial('rectangle', 2000, 0.3, cross_section, stiffAxis=elastic_rods.StiffAxis.D1))
    l.set_holdClosestPointsFixed(True);
    l.set_attraction_tgt_joint_weight(0.01);
    l.attraction_weight = 100;
    return l

### Compute Equilibrium

Similarly there are two different types of viewer depending on whether the surface is visualized.

In [8]:
def get_linkage_eqm(l, opt, cam_param = default_camera_parameters, target_surf = None):
    elastic_rods.compute_equilibrium(l, options = opt)
    if (target_surf is None):
        view = linkage_vis.LinkageViewer(l, width=1024, height=640)
    else:
        view = linkage_vis.LinkageViewerWithSurface(l, target_surf, width=1024, height=640)
    view.setCameraParams(cam_param)
    return l, view

### Comparing solvers

In [9]:
def compare_solvers(pk, po, ok, oo, param_tol=1e-4, obj_tol=1e-4):
    p_diff = abs(pk - po)
    o_diff = abs(oo - ok)
    suspicious = 0
    for i,item in enumerate(p_diff):
        if item > param_tol:
            suspicious = 1
            print("Significant difference for parameter:", i , item, pk[i], po[i])
    if o_diff > obj_tol:
        suspicious = 1
        print("Significant difference for objective values:", ok, oo, o_diff)
    if not suspicious:
        print("No significant differences for parameters and objective values")

# Curved Ribbon Optimization

Initialize the linkage. Currently there are two types of design parameters: rest lengths and rest curvatures. The user can choose to activate or deactivate each one. 

In [10]:
with so(): 
    curved_linkage = initialize_linkage(surface_path = INPUT_SURFACE_PATH,
                                        useCenterline = True, 
                                        model_path = MODEL_PATH, 
                                        cross_section = RIBBON_CS, 
                                        subdivision_res = SUBDIVISION_RESOLUTION)
curved_linkage.set_design_parameter_config(use_restLen = True, use_restKappa = True)
curved_save_tgt_joint_pos = curved_linkage.jointPositions();
curved_linkage_view = linkage_vis.LinkageViewer(curved_linkage)

In [11]:
curved_linkage_view.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

In [12]:
with so(): 
    curved_linkage2 = initialize_linkage(surface_path=INPUT_SURFACE_PATH,
                                         useCenterline=True, 
                                         model_path=MODEL_PATH, 
                                         cross_section=RIBBON_CS, 
                                         subdivision_res=SUBDIVISION_RESOLUTION)
curved_linkage2.set_design_parameter_config(use_restLen=True, use_restKappa=True)
curved_save_tgt_joint_pos2 = curved_linkage2.jointPositions();
curved_linkage_view2 = linkage_vis.LinkageViewer(curved_linkage2)

In [13]:
curved_linkage_view2.show()

Renderer(camera=PerspectiveCamera(children=(PointLight(color='white', intensity=0.6, position=(0.0, 0.0, 5.0),…

## Stage 1

Callback function for updating the viewer during design parameter solve (Stage 1)

In [14]:
def curved_callback(prob, i):
    if (i % 20) != 0: return
    curved_linkage_view.update()
    
def curved_callback2(prob, i):
    if (i % 20) != 0: return
    curved_linkage_view2.update()

In [15]:
curved_dpo = elastic_rods.get_designParameter_optimizer(curved_linkage, rw, sw, callback=curved_callback)
curved_dpo.options.niter = 10000
curved_dpp = curved_dpo.get_problem()

curved_dpo2 = elastic_rods.get_designParameter_optimizer(curved_linkage2, rw, sw, callback=curved_callback2)
curved_dpo2.options.niter = 10000
curved_dpp2 = curved_dpo2.get_problem()

Because how the weights work, this following line should not be run a second time.

In [16]:
with so(): curved_cr = curved_dpo.optimize()

with so(): curved_cr2 = curved_dpo2.optimize()

### Equilibrium solve

In [17]:
with so(): elastic_rods.compute_equilibrium(curved_linkage, options = OPTS)
curved_linkage_view.update()

In [18]:
with so(): elastic_rods.compute_equilibrium(curved_linkage2, options = OPTS)
curved_linkage_view2.update()

## Stage 2

`linkage_optimization` is defined in `python_bindings/linkage_optimization.cc`. The `LinkageOptimization.cc .hh` files are for the XShell (the naming is confusing but this is the unmerged stage. We will unify this after the new solver is in place.

In [19]:
import linkage_optimization

### Initialize the optimizers

In [20]:
# Optimizer for Knitro
OPTS.niter = 25
useCenterline = True
curved_optimizer = linkage_optimization.WeavingOptimization(curved_linkage, 
                                                            INPUT_SURFACE_PATH, 
                                                            useCenterline, 
                                                            equilibrium_options=OPTS, 
                                                            pinJoint=0, 
                                                            useFixedJoint=False)
curved_optimizer.set_target_joint_position(curved_save_tgt_joint_pos)
curved_linkage_view.update()

In [21]:
# Optimizer copy for Optizelle
curved_optimizer2 = linkage_optimization.WeavingOptimization(curved_linkage2, 
                                                             INPUT_SURFACE_PATH, 
                                                             useCenterline, 
                                                             equilibrium_options=OPTS, 
                                                             pinJoint=0, 
                                                             useFixedJoint=False)
curved_optimizer2.set_target_joint_position(curved_save_tgt_joint_pos2)
curved_linkage_view2.update()
OPTS.niter = 200

In [22]:
curved_optimizer.rl_regularization_weight = 0.1
curved_optimizer.smoothing_weight = 10
curved_optimizer.beta = 500000.0
curved_optimizer.gamma = 1

curved_optimizer2.rl_regularization_weight = 0.1
curved_optimizer2.smoothing_weight = 10
curved_optimizer2.beta = 500000.0
curved_optimizer2.gamma = 1

In [23]:
algorithm = linkage_optimization.WeavingOptAlgorithm.NEWTON_CG
def curved_update_viewer():
    curved_linkage_view.update()

We need to run the stage 2 optimizer first with high surface attraction weight.

Define a few params

In [24]:
trust_region_radius = 1.0
tolerance = 1e-2

### Optimize with Knitro

In [25]:
itermax = 2000
curved_optimizer.WeavingOptimize(algorithm, itermax, trust_region_radius, tolerance, curved_update_viewer, minRestLen)


### Optimize with Optizelle

In [26]:
itermax = 100
intpoint_barrier_height = 0.7
tolerance /= 100 # Optizelle needs a smaller tolerance for giving the same results
curved_optimizer2.WeavingOptimizeOptizelle(algorithm, itermax, trust_region_radius, tolerance, intpoint_barrier_height, minRestLen)


### Compare 1st results between KNITRO and OPTIZELLE

In [27]:
# Extract solutions and objective functions values
param_knitro, obj_knitro = np.asarray(curved_optimizer.m_final_params), curved_optimizer.m_final_objective
param_optizelle, obj_optizelle = np.asarray(curved_optimizer2.m_final_params), curved_optimizer2.m_final_objective

curved_linkage_view.update()
curved_linkage_view2.update()

compare_solvers(param_knitro, param_optizelle, obj_optizelle, obj_knitro, param_tol=1e-3, obj_tol=1e-5)

No significant differences for parameters and objective values


## 2nd Optimization 

Then we lower this weight and allow the closest point projections to change.

In [28]:
# Knitro
curved_optimizer.setLinkageAttractionWeight(1e-5)
curved_optimizer.set_holdClosestPointsFixed(False)

# Optizelle

curved_optimizer2.setLinkageAttractionWeight(1e-5)
curved_optimizer2.set_holdClosestPointsFixed(False)


In [29]:
trust_region_radius = 1.0
tolerance = 1e-2

In [30]:
# Knitro 
itermax = 2000
curved_optimizer.WeavingOptimize(algorithm, itermax, trust_region_radius, tolerance, curved_update_viewer, minRestLen)

In [31]:
# Optizelle
itermax = 2000
intpoint_barrier_height = 0.7
tolerance /= 1000 # Optizelle needs a smaller tolerance for giving the same results
curved_optimizer2.WeavingOptimizeOptizelle(algorithm, itermax, trust_region_radius, tolerance, intpoint_barrier_height, minRestLen)

### Compare 2nd results between KNITRO and OPTIZELLE

In [32]:
# Extract solutions and objective functions values
param_knitro, obj_knitro = np.asarray(curved_optimizer.m_final_params), curved_optimizer.m_final_objective
param_optizelle, obj_optizelle = np.asarray(curved_optimizer2.m_final_params), curved_optimizer2.m_final_objective

compare_solvers(param_knitro, param_optizelle, obj_optizelle, obj_knitro, param_tol=1e-3, obj_tol=1e-7)

Significant difference for parameter: 349 0.001013356293286423 -0.010546935853274762 -0.011560292146561185
Significant difference for parameter: 350 0.0011518832828274414 -0.01031152294418539 -0.011463406227012831
Significant difference for parameter: 351 0.0012689265660996481 -0.010035349326756204 -0.011304275892855853
Significant difference for parameter: 352 0.0013593497704430125 -0.00972350401695572 -0.011082853787398733
Significant difference for parameter: 353 0.001418339958433214 -0.00938138899740154 -0.010799728955834755
Significant difference for parameter: 354 0.0014420905122852733 -0.009014627114392016 -0.010456717626677289
Significant difference for parameter: 355 0.00142816082131619 -0.008628968644123632 -0.010057129465439822
Significant difference for parameter: 356 0.0013759504466404945 -0.008230154833236325 -0.00960610527987682
Significant difference for parameter: 357 0.001287205080285148 -0.007823743681452673 -0.009110948761737821
Significant difference for parameter:

### Validate whether the surface attraction weight is set low enough.

In [33]:
curved_optimizer_energy = curved_linkage.energy()
validation_curved_linkage = curved_optimizer.getLinesearchWeaverLinkage()
validation_curved_linkage.attraction_weight = 1e-7
with so(): elastic_rods.compute_equilibrium(validation_curved_linkage, options = OPTS)
validation_curved_view = linkage_vis.LinkageViewer(validation_curved_linkage, width=1024, height=640)
validation_curved_energy = validation_curved_linkage.energy()

print("knitro curved opt energy        ", curved_optimizer_energy)
print("knitro validation curverd energy", validation_curved_energy)
print(abs((validation_curved_energy-curved_optimizer_energy)/curved_optimizer_energy))

curved_optimizer_energy2 = curved_linkage2.energy()
validation_curved_linkage2 = curved_optimizer2.getLinesearchWeaverLinkage()
validation_curved_linkage2.attraction_weight = 1e-7
with so(): elastic_rods.compute_equilibrium(validation_curved_linkage2, options = OPTS)
validation_curved_view = linkage_vis.LinkageViewer(validation_curved_linkage2, width=1024, height=640)
validation_curved_energy2 = validation_curved_linkage2.energy()

print("optizelle curved opt energy       ", curved_optimizer_energy2)
print("optizelle validation curved energy", validation_curved_energy2)
print(abs((validation_curved_energy2-curved_optimizer_energy2)/curved_optimizer_energy2))

knitro curved opt energy         0.0035915395205534744
knitro validation curverd energy 0.003591539510311913
2.851579788139453e-09
optizelle curved opt energy        0.003593381885737765
optizelle validation curved energy 0.003591440009208665
0.0005404036060868441


In [34]:
curved_linkage_view.update()
curved_linkage_view2.update()